In [1]:
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
import os
import random
import uuid
import logging
from dotenv import load_dotenv

# -----------------------
# LOGGING
# -----------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    datefmt="%H:%M:%S",
)
logger = logging.getLogger("ETL_NEO4J")

# -----------------------
# CONFIG
# -----------------------
trace_id = str(uuid.uuid4())[:8]

load_dotenv(dotenv_path="../.env")

uri = os.getenv("NEO4J_URI")
if uri and "neo4j" in uri and "localhost" not in uri:
    logger.warning("⚠️ Detectado entorno local: Cambiando host 'neo4j' por 'localhost'")
    uri = uri.replace("neo4j", "localhost")

user = os.getenv("NEO4J_USER")
password = os.getenv("NEO4J_PASSWORD")
AUTH = (user, password)

logger.info(f"🔌 ({trace_id}) Conectando a: {uri}")
model = SentenceTransformer("all-MiniLM-L6-v2")
logger.info(f"✅ ({trace_id}) Modelo embeddings listo.")

# --- DATASET PRODUCTOS ---
catalog_data = [
    {"id": "L1", "nombre": "MacBook Air M2", "desc": "Laptop ligera Apple chip M2 13 pulgadas", "cat": "Laptops", "precio": 1200},
    {"id": "L2", "nombre": "Dell XPS 13", "desc": "Ultrabook Windows pantalla InfinityEdge", "cat": "Laptops", "precio": 1100},
    {"id": "L3", "nombre": "Asus ROG Zephyrus", "desc": "Laptop Gaming potente RTX 4060", "cat": "Laptops", "precio": 1600},
    {"id": "L4", "nombre": "Lenovo ThinkPad X1", "desc": "Empresarial ultrarresistente fibra de carbono", "cat": "Laptops", "precio": 1400},
    {"id": "A1", "nombre": "Logitech MX Master 3S", "desc": "Mouse ergonómico productividad", "cat": "Accesorios", "precio": 100},
    {"id": "A2", "nombre": "Razer DeathAdder", "desc": "Mouse gaming alta precisión", "cat": "Accesorios", "precio": 60},
    {"id": "M1", "nombre": "Monitor LG Ultrawide", "desc": "Monitor 34 pulgadas curvo trabajo", "cat": "Monitores", "precio": 400},
    {"id": "H1", "nombre": "Sony WH-1000XM5", "desc": "Audífonos cancelación de ruido", "cat": "Audio", "precio": 350},
    {"id": "C1", "nombre": "Cargador Anker 100W", "desc": "Cargador rápido USB-C multipuerto", "cat": "Accesorios", "precio": 50},
]

relationships = [
    ("L1", "A1"), ("L1", "M1"), ("L1", "H1"), ("L1", "C1"),
    ("L2", "A1"), ("L2", "M1"), ("L2", "C1"),
    ("L3", "A2"), ("L3", "M1"), ("L3", "H1"),
]

# --- TIENDAS con CONTACTO ---
tiendas_data = [
    {
        "nombre": "Tienda Central",
        "canal": "fisica",
        "telefono": "+593 99 111 1111",
        "whatsapp": "+593 99 111 1111",
        "direccion": "Av. Principal 123, Centro",
        "horario": "Lun-Sab 09:00-18:00"
    },
    {
        "nombre": "Sucursal Norte",
        "canal": "fisica",
        "telefono": "+593 98 222 2222",
        "whatsapp": "+593 98 222 2222",
        "direccion": "Calle Norte 45, Sector Norte",
        "horario": "Lun-Vie 10:00-19:00"
    },
    {
        "nombre": "Venta Online",
        "canal": "online",
        "telefono": "+593 97 333 3333",
        "whatsapp": "+593 97 333 3333",
        "direccion": "Atención virtual",
        "horario": "Todos los días 08:00-22:00"
    },
]

def setup_database():
    driver = GraphDatabase.driver(uri, auth=AUTH)

    with driver.session() as session:
        # ------------------ FASE ETL 1: Limpieza ------------------
        logger.info(f"🧩 [FASE ETL 1] ({trace_id}) Limpiando base...")
        session.run("MATCH (n) DETACH DELETE n")

        # ------------------ FASE ETL 2: Índices/Constraints ------------------
        logger.info(f"🧩 [FASE ETL 2] ({trace_id}) Creando índices/constraints...")

        # Vector index productos
        try:
            session.run("DROP INDEX productos_embeddings IF EXISTS")
        except Exception as e:
            logger.warning(f"⚠️ ({trace_id}) No se pudo dropear índice (puede ser normal): {e}")

        session.run("""
            CREATE VECTOR INDEX productos_embeddings IF NOT EXISTS
            FOR (p:Producto) ON (p.embedding)
            OPTIONS {
              indexConfig: {
                `vector.dimensions`: 384,
                `vector.similarity_function`: 'cosine'
              }
            }
        """)

        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (p:Producto) REQUIRE p.id IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (t:Tienda) REQUIRE t.nombre IS UNIQUE")
        session.run("CREATE CONSTRAINT IF NOT EXISTS FOR (c:Categoria) REQUIRE c.nombre IS UNIQUE")

        # ------------------ FASE ETL 3: Productos ------------------
        logger.info(f"🧩 [FASE ETL 3] ({trace_id}) Insertando productos...")
        for prod in catalog_data:
            text = f"{prod['nombre']} {prod['desc']} {prod['cat']}"
            vector = model.encode(text).tolist()

            query = """
            MERGE (c:Categoria {nombre: $cat})
            CREATE (p:Producto {id: $id, nombre: $nombre, descripcion: $desc, precio: $precio})
            CREATE (p)-[:PERTENECE_A]->(c)
            SET p.embedding = $vector
            """
            session.run(query, prod, vector=vector)

        # ------------------ FASE ETL 4: Tiendas con contacto ------------------
        logger.info(f"🧩 [FASE ETL 4] ({trace_id}) Creando tiendas con contacto...")
        for t in tiendas_data:
            session.run("""
                MERGE (ti:Tienda {nombre: $nombre})
                SET ti.canal = $canal,
                    ti.telefono = $telefono,
                    ti.whatsapp = $whatsapp,
                    ti.direccion = $direccion,
                    ti.horario = $horario
            """, **t)

        # ------------------ FASE ETL 5: Stock ------------------
        logger.info(f"🧩 [FASE ETL 5] ({trace_id}) Asignando stock a tiendas...")
        for t in tiendas_data:
            for prod in catalog_data:
                cantidad = random.randint(0, 10)
                if cantidad > 0:
                    session.run("""
                        MATCH (ti:Tienda {nombre: $tienda})
                        MATCH (p:Producto {id: $pid})
                        MERGE (ti)-[:TIENE_STOCK {cantidad: $cant}]->(p)
                    """, tienda=t["nombre"], pid=prod["id"], cant=cantidad)

        # ------------------ FASE ETL 6: Compatibilidad ------------------
        logger.info(f"🧩 [FASE ETL 6] ({trace_id}) Creando compatibilidades...")
        for orig, dest in relationships:
            session.run("""
                MATCH (a:Producto {id: $pid1}), (b:Producto {id: $pid2})
                MERGE (a)-[:COMPATIBLE_CON]->(b)
            """, pid1=orig, pid2=dest)

    driver.close()
    logger.info(f"✅ ({trace_id}) ETL finalizado con éxito.")

if __name__ == "__main__":
    setup_database()


c:\Users\karen\Documents\6_ciclo\Modelos_Estocasticos\grafo-conocimiento\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
19:05:36 | WARNING | ETL_NEO4J | ⚠️ Detectado entorno local: Cambiando host 'neo4j' por 'localhost'
19:05:36 | INFO | ETL_NEO4J | 🔌 (6f231832) Conectando a: bolt://localhost:7687
19:05:36 | INFO | sentence_transformers.SentenceTransformer | Use pytorch device_name: cpu
19:05:36 | INFO | sentence_transformers.SentenceTransformer | Load pretrained SentenceTransformer: all-MiniLM-L6-v2
19:05:37 | INFO | httpx | HTTP Request: HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/modules.json "HTTP/1.1 307 Temporary Redirect"
19:05:38 | INFO | httpx | HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/sentence-transformers/all-MiniLM-L6-v2/c9745ed1d9f207